# Cleaning Up Scraped Data

After scraping, only the appropriate tutorial is kept. The selection done by manually deleting tutorials from the `posts.tsv` file. That means there will be some redundant data in the junction table with materials (`posts_ingredients.csv`) and the material itself (`ingredients.csv`). Also there are some mistake when parsing the slug and the materials from the web.

So this notebook will:
- Cleanup redundant data
- Cleanup the text data

In [1]:
import string

import pandas as pd

In [2]:
ins_df = pd.read_csv(r'..\data\posts.tsv', sep='\t')
ins_mat_df = pd.read_csv(r'..\data\posts_ingredients.csv')
mat_df = pd.read_csv(r'..\data\ingredients.csv')

In [21]:
mat_df.rename(columns={'id': 'ingredient_id'}, inplace=True)
ins_mat_df.rename(columns={'id': 'post_ingredients_id'}, inplace=True)

## Cleanup Redundant Data

In [23]:
common = ins_df.merge(ins_mat_df, on=['post_id'])
common.head()

,post_id,title,slug,content,thumbnail,num_of_likes,created_at,deleted_at,user_id,post_ingredients_id,ingredient_id
0,P83810,Making Useful Plant Pot From Waste Packaging M...,making-useful-plant-pot-from-waste-packaging-m...,"<img src=""https://content.instructables.com/FV...",https://content.instructables.com/FV4/9EYY/LHD...,0,15-05-2023 23:59:20,NaN,admin,PM03279,M14593
1,P83810,Making Useful Plant Pot From Waste Packaging M...,making-useful-plant-pot-from-waste-packaging-m...,"<img src=""https://content.instructables.com/FV...",https://content.instructables.com/FV4/9EYY/LHD...,0,15-05-2023 23:59:20,NaN,admin,PM36049,M97197
2,P83810,Making Useful Plant Pot From Waste Packaging M...,making-useful-plant-pot-from-waste-packaging-m...,"<img src=""https://content.instructables.com/FV...",https://content.instructables.com/FV4/9EYY/LHD...,0,15-05-2023 23:59:20,NaN,admin,PM29257,M32099
3,P83810,Making Useful Plant Pot From Waste Packaging M...,making-useful-plant-pot-from-waste-packaging-m...,"<img src=""https://content.instructables.com/FV...",https://content.instructables.com/FV4/9EYY/LHD...,0,15-05-2023 23:59:20,NaN,admin,PM96531,M18290
4,P83810,Making Useful Plant Pot From Waste Packaging M...,making-useful-plant-pot-from-waste-packaging-m...,"<img src=""https://content.instructables.com/FV...",https://content.instructables.com/FV4/9EYY/LHD...,0,15-05-2023 23:59:20,NaN,admin,PM88697,M13435


In [27]:
ins_mat_df_reduced = ins_mat_df[ins_mat_df['post_id'].isin(common['post_id'])]

In [28]:
mat_common = ins_mat_df_reduced.merge(mat_df, on=['ingredient_id'])
mat_common

,post_ingredients_id,post_id,ingredient_id,ingredient,created_at
0,PM03279,P83810,M14593,packaging material,15-05-2023 23:59:20
1,PM36049,P83810,M97197,thermocoal,15-05-2023 23:59:20
2,PM29257,P83810,M32099,paints,15-05-2023 23:59:20
3,PM96531,P83810,M18290,brush,15-05-2023 23:59:20
4,PM88697,P83810,M13435,dry leafs,15-05-2023 23:59:20
...,...,...,...,...,...
506,PM44397,P11813,M97854,acrylic or clear glass,16-05-2023 03:34:47
507,PM61386,P11813,M15096,waste sheet,16-05-2023 03:34:47
508,PM98849,P09869,M18450,pla plastic,16-05-2023 03:35:12
509,PM88648,P09869,M29572,silicon baking mat,16-05-2023 03:35:12


In [29]:
mat_df_reduced = mat_df[mat_df['ingredient_id'].isin(mat_common['ingredient_id'])]

## Cleanup Text

### Posts' Slug

In [44]:
import string
import re
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [45]:
def create_refined_slug(title: str):
    clean_title = title.translate(str.maketrans('', '', string.punctuation))
    clean_title = re.sub(' +', ' ', clean_title)
    return clean_title.lower().replace(' ', '-')

In [46]:
ins_df['slug'] = ins_df['title'].apply(create_refined_slug)

In [47]:
ins_df

,post_id,title,slug,content,thumbnail,num_of_likes,created_at,deleted_at,user_id
0,P83810,Making Useful Plant Pot From Waste Packaging M...,making-useful-plant-pot-from-waste-packaging-m...,"<img src=""https://content.instructables.com/FV...",https://content.instructables.com/FV4/9EYY/LHD...,0,15-05-2023 23:59:20,NaN,admin
1,P03905,Handmade Solstice Clock (Best Out of Waste),handmade-solstice-clock-best-out-of-waste,"<img src=""https://content.instructables.com/FP...",https://content.instructables.com/FP8/JQAG/LEO...,0,16-05-2023 00:00:05,NaN,admin
2,P30512,DIY Best of Waste,diy-best-of-waste,"<img src=""https://content.instructables.com/FG...",https://content.instructables.com/FGC/0OGY/LEL...,0,16-05-2023 00:00:26,NaN,admin
3,P79840,A Bluetooth Speaker Built With Waste Material,a-bluetooth-speaker-built-with-waste-material,"<img src=""https://content.instructables.com/FL...",https://content.instructables.com/FLR/949N/LE1...,0,16-05-2023 00:00:49,NaN,admin
4,P30021,3D WASTE BLUETOOTH,3d-waste-bluetooth,"<img src=""https://content.instructables.com/F4...",https://content.instructables.com/F4X/VNDM/LDZ...,0,16-05-2023 00:01:49,NaN,admin
5,P74341,Glowing Light From Packaging Waste,glowing-light-from-packaging-waste,"<img src=""https://content.instructables.com/FP...",https://content.instructables.com/FP4/47SN/F23...,0,16-05-2023 00:02:41,NaN,admin
6,P50019,How to Keep Warm on Waste Heat,how-to-keep-warm-on-waste-heat,"<img src=""https://content.instructables.com/FX...",https://content.instructables.com/FXK/O317/FPW...,0,16-05-2023 00:03:54,NaN,admin
7,P09071,Acetone Recycler (Industrial Waste Management),acetone-recycler-industrial-waste-management,"<img src=""https://content.instructables.com/F5...",https://content.instructables.com/F5L/1L7B/FTD...,0,16-05-2023 00:05:05,NaN,admin
8,P14029,Recycled Computer Monitor to Waste Basket (sup...,recycled-computer-monitor-to-waste-basket-supe...,"<img src=""https://content.instructables.com/FR...",https://content.instructables.com/FRC/YIUX/GJQ...,0,16-05-2023 00:07:01,NaN,admin
9,P05014,"""Toxic Waste"" Halloween Costume!",toxic-waste-halloween-costume,"<img src=""https://content.instructables.com/F5...",https://content.instructables.com/F53/JA8Y/GTO...,0,16-05-2023 03:00:40,NaN,admin


## Export the cleaned data

In [48]:
ins_df.to_csv(r'..\data\posts.tsv', index=False, sep='\t')
ins_mat_df.to_csv(r'..\data\posts_ingredients.csv', index=False)
mat_df.to_csv(r'..\data\ingredients.csv', index=False)